# 문서군집화 - Opinion Review dataset

텍스트 문서도 군집화 할 수 있어. 어찌나 계산의 영역이기 때문이지.   
데이터가 파일단위로 되어있는것은 어떻게 처리할 것인지. 

In [4]:
import pandas as pd
import glob, os  # 글로브와 os 등장

path = 'OpinosisDataset1.0/topics'
os.path.join(path, '*data')       # Windows - // , Mac - /  이렇게 나와. 

'OpinosisDataset1.0/topics\\*data'

In [5]:
all_files = glob.glob(os.path.join(path, '*.data'))
all_files[:5]
# 반복문 사용할 수 있어.. os 로 패스를 쓰면 

['OpinosisDataset1.0/topics\\accuracy_garmin_nuvi_255W_gps.txt.data',
 'OpinosisDataset1.0/topics\\bathroom_bestwestern_hotel_sfo.txt.data',
 'OpinosisDataset1.0/topics\\battery-life_amazon_kindle.txt.data',
 'OpinosisDataset1.0/topics\\battery-life_ipod_nano_8gb.txt.data',
 'OpinosisDataset1.0/topics\\battery-life_netbook_1005ha.txt.data']

In [6]:
file = all_files[0]
file

'OpinosisDataset1.0/topics\\accuracy_garmin_nuvi_255W_gps.txt.data'

In [7]:
file.split('\\')[-1].split('.')[0]
# 확장자 걷어내고 파일 이름을 가져오고 싶을때 

'accuracy_garmin_nuvi_255W_gps'

encoding 에러 관련
latin1
f = open(path, 'r')    --->    f = open(path, 'r', encoding='UTF-8') 
출처: https://nackwon.tistory.com/120 [JIMMY]

UnicodeDecodeError: 'cp949' codec can't decode byte 0xa3 in position 8506: illegal multibyte sequence 유니코드 에러 어떻게 수정 
'rt', encoding='UTF8'

In [9]:
filename_list = []
opinion_text = []
for file in glob.glob(os.path.join(path, '*.data')):
    with open(file, 'r', encoding='latin1') as f:
        text = f.read()
    opinion_text.append(text)
    filename = file.split('\\')[-1].split('.')[0]
    filename_list.append(filename)

df = pd.DataFrame({'filename':filename_list, 'opinion':opinion_text})
df.head(3)

,filename,opinion
0,accuracy_garmin_nuvi_255W_gps,", and is very, very accurate .\n but for the m..."
1,bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and ve..."
2,battery-life_amazon_kindle,After I plugged it in to my USB hub on my com...


# 글로브 . 글로브의 빈번한 사용

with 설명중   

유닉스 리눅스 에서   

늘 파일을 open()

read(), write()

close() 

순서로 하도록 만들었는데, 파일에서 읽은후 클로즈를 않하면. 시스템의 리소스가 계속 끌고끌고가 
파이썬의 방법은 open을 with (for같이)뜨면 윗드 문을 벗어나는 순간에 파일이 자동 클로즈 

with open() as f:  리소스 낭비를 

- simple tokenizer 함수를 이용해 feature 변환

In [10]:
from nltk import word_tokenize

def simple_tokenizer(text):       # 글자수 2개 이하인 토큰만 제거 
    return [word for word in word_tokenize(text) if len(word) > 2]

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

tvect = TfidfVectorizer(tokenizer=simple_tokenizer, stop_words='english',
                        ngram_range=(1,2), min_df=0.05, max_df=0.85)
feature = tvect.fit_transform(df.opinion)

- 군집화

In [12]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, max_iter=10000, random_state=2022)
kmeans.fit(feature)

KMeans(max_iter=10000, n_clusters=5, random_state=2022)

In [13]:
df['cluster'] = kmeans.labels_
df.head()

,filename,opinion,cluster
0,accuracy_garmin_nuvi_255W_gps,", and is very, very accurate .\n but for the m...",0
1,bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and ve...",2
2,battery-life_amazon_kindle,After I plugged it in to my USB hub on my com...,1
3,battery-life_ipod_nano_8gb,short battery life I moved up from an 8gb .\...,1
4,battery-life_netbook_1005ha,"6GHz 533FSB cpu, glossy display, 3, Cell 23Wh ...",1


In [14]:
df.cluster.value_counts()

2    16
3    13
4    10
0     7
1     5
Name: cluster, dtype: int64

3개의 집합으로 군집화

In [15]:
# 3개의 집합으로 군집화
kmeans = KMeans(n_clusters=3, max_iter=10000, random_state=2022)
kmeans.fit(feature)
df['cluster_label'] = kmeans.labels_
df.cluster_label.value_counts()

0    25
1    16
2    10
Name: cluster_label, dtype: int64

- 군집별 핵심 단어 추출하기

In [16]:
feature.shape

(51, 4154)

In [17]:
cluster_centers = kmeans.cluster_centers_
cluster_centers.shape

(3, 4154)

- 클러스터 파이 코드 공부할것

In [18]:
from cluster import get_cluster_details

feature_names = tvect.get_feature_names()
cluster_details = get_cluster_details(cluster_model=kmeans, cluster_data=df,
                                      feature_names=feature_names, clusters_num=3, top_n_features=10)

In [19]:
for cluster_num, cluster_detail in cluster_details.items():
    print(f'####### Cluster {cluster_num}')
    print('Top features:', cluster_detail['top_features'])
    print('Reviews 파일명:', cluster_detail['filenames'][:7])
    print('==================================================')

####### Cluster 0
Top features: ['screen', 'battery', 'battery life', 'keyboard', 'kindle', 'life', 'directions', 'size', 'voice', 'speed']
Reviews 파일명: ['accuracy_garmin_nuvi_255W_gps', 'battery-life_amazon_kindle', 'battery-life_ipod_nano_8gb', 'battery-life_netbook_1005ha', 'buttons_amazon_kindle', 'directions_garmin_nuvi_255W_gps', 'display_garmin_nuvi_255W_gps']
####### Cluster 1
Top features: ['hotel', 'service', 'rooms', 'staff', 'room', 'food', 'location', 'clean', 'bathroom', 'parking']
Reviews 파일명: ['bathroom_bestwestern_hotel_sfo', 'food_holiday_inn_london', 'food_swissotel_chicago', 'free_bestwestern_hotel_sfo', 'location_bestwestern_hotel_sfo', 'location_holiday_inn_london', 'parking_bestwestern_hotel_sfo']
####### Cluster 2
Top features: ['interior', 'mileage', 'seats', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
Reviews 파일명: ['comfort_honda_accord_2008', 'comfort_toyota_camry_2007', 'gas_mileage_toyota_camry_2007', 'interior_hond